In [3]:
import cv2
import os
from imageio.v2 import imread
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [5]:
data_folders = ["Cyst", "Stone", "Tumor", "Normal"]
flat_data_arr = []
target_arr = []
base_dir = 'Dataset'
output_dir = 'Preprocessed'

In [7]:
import numpy as np

def clahe(image_path):
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gray_image)
    return clahe_image

def gaussian_blur(image):
    #image = cv2.imread(image_path)
    kernel_size = (5, 5)
    sigma = 0
    filtered_image = cv2.GaussianBlur(image, kernel_size, sigma)
    return filtered_image

def find_contour(image):
# Preprocessing
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    _, binary_image = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Contour detection
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area or other criteria if necessary
    # For example, you can iterate through contours and filter based on contour area:
    # filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Draw contours on original image
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
    contour_image = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 2)
    return contour_image



def watershed_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labeling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Mark the unknown region with zero
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    markers = cv2.watershed(image, markers)

    # Overlay segmentation result on original image
    image[markers == -1] = [255, 0, 0]  # Mark watershed boundaries

    return image


for folder in data_folders:
    folder_path = os.path.join(base_dir, folder)
    print("Folder Path:", folder_path)
    output_folder = os.path.join(output_dir, folder)
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            # Preprocess image
            preprocessed_image = clahe(image_path)
            preprocessed_image = gaussian_blur(preprocessed_image)
            preprocessed_image = find_contour(preprocessed_image)
            preprocessed_image = watershed_segmentation(preprocessed_image )

            # Save preprocessed image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, preprocessed_image)
print('Preprocess Completed')

Folder Path: Dataset\Cyst
Folder Path: Dataset\Stone
Folder Path: Dataset\Tumor
Folder Path: Dataset\Normal
Preprocess Completed


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths for your data
data_dir = 'Preprocessed'
train_dir = 'TRAIN'
test_dir = 'TEST'
val_dir = 'VAL'

# Define your categories or classes
categories = ["Cyst", "Stone", "Tumor", "Normal"]

# Create train, test, and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split the data into train, test, and validation sets
for category in categories:
    category_dir = os.path.join(data_dir, category)
    files = os.listdir(category_dir)
    train_files, test_val_files = train_test_split(files, test_size=0.3, random_state=42)
    test_files, val_files = train_test_split(test_val_files, test_size=0.5, random_state=42)

    # Move files to appropriate directories
    for file in train_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(train_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for file in test_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(test_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for file in val_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(val_dir, category)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)


In [8]:
import numpy as np
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
init_notebook_mode(connected=True)
RANDOM_SEED = 123

ModuleNotFoundError: No module named 'plotly'